ใน ep นี้เราจะมาเรียนรู้เทคนิคการใช้งาน Proximity Analysis กันต่อ เราจะสมมติตัวเองว่าอยู่ในทีม Crisis Response ที่จะวิเคราะห์อุบัติเหตุทางถนน รถชน และการรับมือเหตุฉุกเฉินของโรงพยาบาล ในเมือง New York City เพื่อตอบคำถามดังนี้

1. มีอุบัติเหตุทางถนนกี่เปอร์เซ็นต์ ที่เกิดห่างจากนอกเขตโรงพยาบาลเกินรัศมี 10 กิโลเมตร?
1. สมมติถ้าเกิดอุบัติเหตุทางถนนขึ้น ควรจะไปโรงพยาบาลไหนที่อยู่พิกัดใกล้ที่สุด
1. โรงพยาบาลไหนน่าจะงานล้น จากอุบัติเหตุนอกเขตของตัวเอง
1. ควรสร้างโรงพยาบาลใหม่ ณ ทำเลแถวไหนของเมือง ที่จะช่วยให้ครอบคลุมอุบัติเหตุทางถนน ดูแลผู้ป่วยมากขึ้นที่สุด เพื่อลดเปอร์เซ็นอุบัติเหตุนอกเขต ให้ต่ำกว่า 10%

# 0. Install

เราจะต้อง Install kaggle เพื่อ Download Dataset, geopandas เพื่อใช้ในการวิเคราะห์ข้อมูล geospatial, folium เพื่อแสดงแผ่นที่ (ถ้ายังไม่ได้ Install ให้ uncomment)

In [0]:
# ! pip install geopandas
# ! pip install git+https://github.com/python-visualization/folium
# ! pip install kaggle --upgrade

# 1. Import Library

Import folium Library เพื่อใช้ในการพล็อตแผนที่แบบ Interactive

In [0]:
import math
import pandas as pd
import geopandas as gpd
from shapely.geometry import MultiPolygon

import folium
from folium import Choropleth, Marker
from folium.plugins import HeatMap, MarkerCluster

import os
from pathlib import Path

ประกาศฟังก์ชันในการแสดงแผนที่โดยใช้ HTML iframe แต่ [Colab](https://www.bualabs.com/archives/1687/what-is-colab-open-jupyter-notebook-in-github-on-google-colab-create-open-in-colab-button-colab-ep-1/) ไม่ Support iframe เราจะ return Map ออกไปเลย

In [0]:
from IPython.display import IFrame, HTML

def embed_map(m, file_name):    
    m.save(file_name)

    # # VM
    # return IFrame(src=file_name, width='100%', height='500px')

    # Colab
    return m

# 2. เตรียม Path สำหรับดาวน์โหลดข้อมูล

กำหนด path ของ Config File และ Dataset ว่าจะอยู่ใน Google Drive ถ้าเราใช้ [Google Colab](https://www.bualabs.com/archives/1687/what-is-colab-open-jupyter-notebook-in-github-on-google-colab-create-open-in-colab-button-colab-ep-1/) หรือ อยู่ใน HOME ถ้าเราใช้ VM ธรรมดา และกำหนด Environment Variable ไปยังโฟลเดอร์ที่เก็บ kaggle.json

ในกรณีใช้ Colab ให้ Mount Google Drive เพื่อดึง Config File มาจาก Google Drive ส่วนตัวของเรา เมื่อเรารัน Cell ด้านล่างจะมีลิงค์ปรากฎขึ้นมาให้เรา Login กด Approve แล้ว Copy Authorization Code มาใส่ในช่องด้านล่าง แล้วกด Enter

In [4]:
dataset = 'alexisbcook/geospatial-learn-course-data'

# Google Colab
config_path = Path('/content/drive')
data_path = Path('/content/datasets/')/dataset
from google.colab import drive
drive.mount(str(config_path))
os.environ['KAGGLE_CONFIG_DIR'] = f"{config_path}/My Drive/.kaggle"

## VM
# config_path = Path(os.getenv("HOME"))
# data_path = config_path/"datasets"/dataset
# data_path.mkdir(parents=True, exist_ok=True)
# os.environ['KAGGLE_CONFIG_DIR'] = f"{config_path}/.kaggle"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 3. Dataset

ในเคสนี้ เราจะสมมติตัวเองเป็นทีม Crisis Response ที่จะวิเคราะห์อุบัติเหตุทางถนน รถชน และการรับมือเหตุฉุกเฉิน บริหารจัดการ แบ่งงานดูแลเคสอุบัติภัยทางถนนที่เกิดขึ้น ของโรงพยาบาล ในเมือง New York City 

Dataset เราจะดึงจาก Kaggle วิธี Download kaggle.json ให้ดูจาก ep ที่แล้ว

เมื่อได้ kaggle.json มาแล้ว ในกรณีใช้ Google Colab ให้นำมาใส่ไว้ในโฟลเดอร์ My Drive/.kaggle ใน Google Drive ของเรา เป็น My Drive/.kaggle/kaggle.json ถ้าใช้ VM ให้ใส่ใน HOME/.kaggle/

สั่งดาวน์โหลด Dataset จาก Kaggle พร้อมทั้ง unzip ไว้ใน data_path

In [5]:
!kaggle datasets download {dataset} -p "{data_path}" --unzip

100% 233M/233M [00:01<00:00, 103MB/s]
100% 233M/233M [00:01<00:00, 133MB/s]


# 4. Data

## 4.1 ข้อมูลภูมิศาสตร์ อุบัติเหตุทางถนน

โหลดข้อมูลอุบัติภัยทางถนน ในเมือง นิวยอร์ค ระหว่างปี  2013-2018.

In [24]:
collisions = gpd.read_file(data_path/"NYPD_Motor_Vehicle_Collisions/NYPD_Motor_Vehicle_Collisions/NYPD_Motor_Vehicle_Collisions.shp")
collisions.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET,CROSS STRE,OFF STREET,NUMBER OF,NUMBER O_1,NUMBER O_2,NUMBER O_3,NUMBER O_4,NUMBER O_5,NUMBER O_6,NUMBER O_7,CONTRIBUTI,CONTRIBU_1,CONTRIBU_2,CONTRIBU_3,CONTRIBU_4,UNIQUE KEY,VEHICLE TY,VEHICLE _1,VEHICLE _2,VEHICLE _3,VEHICLE _4,geometry
0,07/30/2019,0:00,BRONX,10464,40.841100,-73.784960,"(40.8411, -73.78496)",None,None,121 PILOT STREET,1.0,0.0,0,0,0,0,1,0,Unspecified,Unspecified,Unspecified,None,None,4180045,Sedan,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,None,None,POINT (1043750.211 245785.815)
1,07/30/2019,0:10,QUEENS,11423,40.710827,-73.770660,"(40.710827, -73.77066)",JAMAICA AVENUE,188 STREET,None,1.0,0.0,0,0,0,0,1,0,Driver Inattention/Distraction,Unspecified,None,None,None,4180007,Sedan,Sedan,None,None,None,POINT (1047831.185 198333.171)
2,07/30/2019,0:25,None,None,40.880318,-73.841286,"(40.880318, -73.841286)",BOSTON ROAD,None,None,1.0,0.0,0,0,0,0,1,0,Following Too Closely,Unspecified,None,None,None,4179575,Sedan,Station Wagon/Sport Utility Vehicle,None,None,None,POINT (1028139.293 260041.178)
3,07/30/2019,0:35,MANHATTAN,10036,40.756744,-73.984590,"(40.756744, -73.98459)",None,None,155 WEST 44 STREET,1.0,0.0,0,0,0,0,1,0,Oversized Vehicle,Unspecified,None,None,None,4179544,Box Truck,Station Wagon/Sport Utility Vehicle,None,None,None,POINT (988519.261 214979.320)
4,07/30/2019,10:00,BROOKLYN,11223,40.600090,-73.965910,"(40.60009, -73.96591)",AVENUE T,OCEAN PARKWAY,None,1.0,0.0,0,0,1,0,0,0,Traffic Control Disregarded,Unspecified,None,None,None,4180660,Station Wagon/Sport Utility Vehicle,Bike,None,None,None,POINT (993716.669 157907.212)


มีอุบัติเหตุ เกิดขึ้น จำนวน 261,905 ครั้ง

In [25]:
collisions.shape

(261905, 30)

เนื่องจาก Colab เจอข้อมูลเยอะ ๆ แล้วอาจจะ Error เราจะ Sample ข้อมูลให้เหลือ 40,000 Record ให้พอเห็นภาพ

In [0]:
# Colab
collisions = collisions.sample(40000)

## 4.2 แผนที่ อุบัติเหตุทางถนน 

พล็อตแผนที่ อุบัติเหตุทางถนน ในเมือง New York City

In [0]:
# Create a base map
m_1 = folium.Map(location=[40.7, -74], zoom_start=11)

# Add a heatmap to the base map
HeatMap(data=collisions[['LATITUDE', 'LONGITUDE']], radius=12).add_to(m_1)

# Display the map
embed_map(m_1, '25h-m_1.html')

<!--- สำหรับแสดงบนเว็บไซต์ -->
<iframe width="100%" height="640" src="https://www.bualabs.com/wp-content/uploads/2019/11/25h-m_1.html" frameborder="0" allowfullscreen></iframe>

## 4.3 ข้อมูลภูมิศาสตร์โรงพยาบาล

ใน Dataset ของเรายังมีข้อมูลที่น่าสนใจอีกอย่าง คือ ข้อมูลโรงพยาบาล ในเมืองเดียวกัน

In [28]:
hospitals = gpd.read_file(data_path/"nyu_2451_34494/nyu_2451_34494/nyu_2451_34494.shp")
hospitals.head()

,id,name,address,zip,factype,facname,capacity,capname,bcode,xcoord,ycoord,latitude,longitude,geometry
0,317000001H1178,BRONX-LEBANON HOSPITAL CENTER - CONCOURSE DIVI...,1650 Grand Concourse,10457,3102,Hospital,415,Beds,36005,1008872.0,246596.0,40.843490,-73.911010,POINT (1008872.000 246596.000)
1,317000001H1164,BRONX-LEBANON HOSPITAL CENTER - FULTON DIVISION,1276 Fulton Ave,10456,3102,Hospital,164,Beds,36005,1011044.0,242204.0,40.831429,-73.903178,POINT (1011044.000 242204.000)
2,317000011H1175,CALVARY HOSPITAL INC,1740-70 Eastchester Rd,10461,3102,Hospital,225,Beds,36005,1027505.0,248287.0,40.848060,-73.843656,POINT (1027505.000 248287.000)
3,317000002H1165,JACOBI MEDICAL CENTER,1400 Pelham Pkwy,10461,3102,Hospital,457,Beds,36005,1027042.0,251065.0,40.855687,-73.845311,POINT (1027042.000 251065.000)
4,317000008H1172,LINCOLN MEDICAL & MENTAL HEALTH CENTER,234 E 149 St,10451,3102,Hospital,362,Beds,36005,1005154.0,236853.0,40.816758,-73.924478,POINT (1005154.000 236853.000)


## 4.4 แผนที่โรงพยาบาล กับบริเวณที่มักเกิดอุบัติเหตุบนท้องถนน

In [0]:
m_2 = folium.Map(location=[40.7, -74], zoom_start=11) 


HeatMap(data=collisions[['LATITUDE', 'LONGITUDE']], radius=12).add_to(m_2)

# Visualize the hospital locations
for idx, row in hospitals.iterrows():    
    Marker([row['latitude'], row['longitude']], popup=row['name']).add_to(m_2)

# Show the map
embed_map(m_2, '25h-m_2.html')

<!--- สำหรับแสดงบนเว็บไซต์ -->
<iframe width="100%" height="640" src="https://www.bualabs.com/wp-content/uploads/2019/11/25h-m_2.html" frameborder="0" allowfullscreen></iframe>

# 5. GeoSpatial Analysis

## 5.1 มีอุบัติเหตุมากแค่ไหน ที่เกิดห่างจากโรงพยาบาลเกิน 10 กิโลเมตร?

เราจะสร้าง Buffer รัศมี 10 กิโลเมตร ขึ้นมารอบ ๆ ทุกโรงพยาบาลในเมือง

In [0]:
coverage = hospitals.geometry.buffer(10*1000)

m_3 = folium.Map(location=[40.7, -74], zoom_start=11) 

HeatMap(data=collisions[['LATITUDE', 'LONGITUDE']], radius=12).add_to(m_3)

for idx, row in hospitals.iterrows():
    Marker([row['latitude'], row['longitude']], popup=row['name']).add_to(m_3)
    
# Plot each polygon on the map
folium.GeoJson(coverage.to_crs(epsg=2263)).add_to(m_3)

# Show the map
embed_map(m_3, '25h-m_3.html')

<!--- สำหรับแสดงบนเว็บไซต์ -->
<iframe width="100%" height="640" src="https://www.bualabs.com/wp-content/uploads/2019/11/25h-m_3.html" frameborder="0" allowfullscreen></iframe>

นำ Coverage มารวมกันเป็น MultiPolygon shape เดียว เพื่อให้ง่ายในการกรอง เอาอุบัติเหตุที่ไม่อยู่ใน shape ออกมา

In [31]:
cov_union = coverage.geometry.unary_union
outside_range = collisions.loc[~collisions["geometry"].apply(lambda x: cov_union.contains(x))]
outside_range.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET,CROSS STRE,OFF STREET,NUMBER OF,NUMBER O_1,NUMBER O_2,NUMBER O_3,NUMBER O_4,NUMBER O_5,NUMBER O_6,NUMBER O_7,CONTRIBUTI,CONTRIBU_1,CONTRIBU_2,CONTRIBU_3,CONTRIBU_4,UNIQUE KEY,VEHICLE TY,VEHICLE _1,VEHICLE _2,VEHICLE _3,VEHICLE _4,geometry
9594,05/14/2019,16:30,QUEENS,11422,40.661263,-73.729320,"(40.661263, -73.72932)",139 AVENUE,254 STREET,None,1.0,0.0,0,0,0,0,1,0,Failure to Yield Right-of-Way,Unspecified,None,None,None,4131829,Station Wagon/Sport Utility Vehicle,Sedan,None,None,None,POINT (1059347.865 180308.360)
121654,09/14/2016,9:40,None,None,40.728718,-73.751380,"(40.728718, -73.75138)",GRAND CENTRAL PKWY,None,None,1.0,0.0,0,0,0,0,1,0,Following Too Closely,Unspecified,None,None,None,3519573,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,SPORT UTILITY / STATION WAGON,None,None,POINT (1053157.771 204865.968)
256633,08/24/2012,18:20,QUEENS,11421,40.693263,-73.853236,"(40.6932632, -73.8532358)",JAMAICA AVENUE,92 STREET,None,1.0,0.0,0,0,1,0,0,0,Unspecified,Unspecified,None,None,None,204136,PASSENGER VEHICLE,BICYCLE,None,None,None,POINT (1024948.992 191885.018)
192529,07/22/2014,16:30,QUEENS,11417,40.682265,-73.845133,"(40.6822652, -73.8451332)",103 AVENUE,WOODHAVEN BOULEVARD,None,1.0,0.0,1,0,0,0,0,0,Unspecified,None,None,None,None,997558,None,None,None,None,None,POINT (1027202.995 187882.000)
160023,08/02/2015,1:20,STATEN ISLAND,10308,40.553555,-74.141164,"(40.5535547, -74.1411639)",AMBOY ROAD,CRANFORD STREET,None,0.0,1.0,0,0,0,0,0,1,Traffic Control Disregarded,Unspecified,None,None,None,3269992,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,None,None,None,POINT (945021.993 140983.012)


ได้จำนวน 6102 ครั้ง จากจำนวนที่เรา Sample ออกมา

In [32]:
outside_range.shape

(6102, 30)

อุบัติเหตุทางถนน ที่อยู่นอกเขต 10 กิโลเมตร จากโรงพยาบาล คิดเป็น 15%

In [33]:
percentage = round(100*len(outside_range)/len(collisions), 2)
print("Percentage of collisions more than 10 km away from the closest hospital: {}%".format(percentage))

Percentage of collisions more than 10 km away from the closest hospital: 15.26%


# 6. Recommender System

เราจะช่วยปรับปรุงให้ดีขึ้นได้อย่างไร

## 6.1 แนะนำโรงพยาบาลที่ใกล้ที่สุด

เมื่อใส่พิกัดที่เกิดอุบัติเหตุเข้าไป ระบบจะแสดงชื่อโรงพยาบาลที่ใกล้ที่สุด

In [34]:
def best_hospital(collision_location):
    # Your code here
    name = hospitals.loc[hospitals.distance(collision_location).idxmin()]['name']
    return name

ST JOHNS EPISCOPAL HOSPITAL SO SHORE


In [0]:
print(best_hospital(outside_range.geometry.iloc[0]))

## 6.2 โรงพยาบาลไหนน่าจะงานล้น

ถ้าพิจารณาเฉพาะอุบัติเหตุที่อยู่นอกเขต 10 กิโลเมตร โรงพยาบาลไหนน่าจะรับ Load มากที่สุด

In [35]:
recommended_hosp = outside_range["geometry"].apply(lambda x: best_hospital(x))
highest_demand = recommended_hosp.value_counts().idxmax()
highest_demand

'JAMAICA HOSPITAL MEDICAL CENTER'

## 6.3 สร้างโรงพยาบาลเพิ่ม แถวไหนดี

เราจะพล็อตแผนที่ คัดเอาแต่อุบัติเหตุทางถนน ที่อยู่นอกเขตความครอบคลุม แล้วลองเลือกจุด จำนวน 2 จุด เป็นที่สร้างโรงพยาบาลใหม่ ว่าจะทำให้ความคลอบคลุมเพิ่มขึ้นกี่เปอร์เซ็น

In [0]:
m_6 = folium.Map(location=[40.7, -74], zoom_start=11) 

folium.GeoJson(coverage.geometry.to_crs(epsg=4326)).add_to(m_6)
HeatMap(data=outside_range[['LATITUDE', 'LONGITUDE']], radius=12).add_to(m_6)
folium.LatLngPopup().add_to(m_6)

embed_map(m_6, '25h-m_6.html')

<!--- สำหรับแสดงบนเว็บไซต์ -->
<iframe width="100%" height="640" src="https://www.bualabs.com/wp-content/uploads/2019/11/25h-m_6.html" frameborder="0" allowfullscreen></iframe>

พิจารณจากแผนที่ด้านบน เราจะลองคลิกลงบนแผนที่ ในย่านที่มีอุบัติเหตุหนาแน่น ที่อยู่นอกวงกลม 2 จุด เพิ่อแสดงพิกัด แล้วเอามาใส่ด้านล่าง เพื่อจุดที่จะสร้างโรงพยาบาลใหม่ ว่าจะทำให้ เปอร์เซ็นอุบัติเหตุที่อยู่นอกเขต ลดลงกี่เปอร์เซ็น

In [0]:
# Proposed location of hospital 1
lat_1 = 40.6754
long_1 = -73.7498

# Proposed location of hospital 2
lat_2 = 40.6779
long_2 = -73.8721

ลองพล็อตแผนที่ หลังจากที่สร้างโรงพยาบาลใหม่ 2 แห่ง ตามพิกัดด้านบน แล้วคำนวนความครอบคลุม เปรียบเทียบกับของเก่า

In [0]:
new_df = pd.DataFrame(
    {'Latitude': [lat_1, lat_2],
        'Longitude': [long_1, long_2]})
new_gdf = gpd.GeoDataFrame(new_df, geometry=gpd.points_from_xy(new_df.Longitude, new_df.Latitude))
new_gdf.crs = {'init' :'epsg:4326'}
new_gdf = new_gdf.to_crs(epsg=2263)
# get new percentage
new_coverage = gpd.GeoDataFrame(geometry=new_gdf.geometry).buffer(10000)
new_my_union = new_coverage.geometry.unary_union
new_outside_range = outside_range.loc[~outside_range["geometry"].apply(lambda x: new_my_union.contains(x))]
new_percentage = round(100*len(new_outside_range)/len(collisions), 2)

# make the map
m_7 = folium.Map(location=[40.7, -74], zoom_start=11) 
folium.GeoJson(coverage.geometry.to_crs(epsg=4326)).add_to(m_7)
folium.GeoJson(new_coverage.geometry.to_crs(epsg=4326)).add_to(m_7)
for idx, row in new_gdf.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=f'New Hospital {idx+1}', icon=folium.Icon(color='orange')).add_to(m_7)
HeatMap(data=new_outside_range[['LATITUDE', 'LONGITUDE']], radius=12).add_to(m_7)
folium.LatLngPopup().add_to(m_7)
display(embed_map(m_7, '25h-m_7.html'))

<!--- สำหรับแสดงบนเว็บไซต์ -->
<iframe width="100%" height="640" src="https://www.bualabs.com/wp-content/uploads/2019/11/25h-m_7.html" frameborder="0" allowfullscreen></iframe>

ถ้าเราสร้างโรงพยาบาลใหม่ 2 โรงในตำแหน่งด้านบน อุบัติเหตุนอกเขต ลดลงเหลือไม่ถึง 10 % เท่านั้น

In [41]:
print("(NEW) Percentage of collisions more than 10 km away from the closest hospital: {}%".format(new_percentage))

(NEW) Percentage of collisions more than 10 km away from the closest hospital: 9.2%


# Credit

* https://www.kaggle.com/alexisbcook/proximity-analysis
* https://epsg.io/2272
* https://www.bualabs.com/archives/2785/find-next-starbucks-reserve-roastery-new-branch-store-location-big-data-analyst-demography-geography-geospatial-ep-6/
* http://geopandas.org/reference.html
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html
* https://python-visualization.github.io/folium/quickstart.html#GeoJSON/TopoJSON-Overlays
* https://epsg.io/4326
* http://geopandas.org/geometric_manipulations.html